In [3]:
library(Seurat)
library(ggplot2)
library(CellChat)
library(BiocParallel)

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: igraph


Attaching package: ‘igraph’


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following object is masked from ‘package:Seurat’:

    components


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union




In [1]:
adata <-schard::h5ad2seurat('batch_corrected_v1.h5ad', use.raw = TRUE)


Loading required package: Seurat

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t




In [2]:
adata <- NormalizeData(adata)

In [5]:
Idents(adata) <- "annot"

In [6]:
adata <- subset(adata, idents = c("Acinar Cells", "B Cells", "Cycling", "Normal Duct"), invert=TRUE)

In [7]:
cond<-SplitObject(adata, split.by = "condition")


In [8]:
runCellChat <- function(subset_ad){
    assay <- GetAssayData(subset_ad, assay = "RNA", slot = "counts")
    assay <- normalizeData(assay)
    Idents(subset_ad) <- "annot"
    labels <- Idents(subset_ad)
    meta <- subset_ad[[]]
    meta$samples <- meta$sample_id
    meta$labels <- meta$annot
    rm(subset_ad)
    cellchat <- createCellChat(object = assay, group.by = "annot", meta=meta)
    CellChatDB <- CellChatDB.mouse
    CellChatDB.use <- subsetDB(CellChatDB, search = "Secreted Signaling")
    cellchat@DB <- CellChatDB.use
    cellchat <- subsetData(cellchat)
    cellchat <- identifyOverExpressedGenes(cellchat)
    cellchat <- identifyOverExpressedInteractions(cellchat)
    cellchat <- computeCommunProb(cellchat, type = "triMean")
    cellchat <- filterCommunication(cellchat, nonFilter.keep=TRUE, rare.keep=TRUE)
    cellchat <- computeCommunProbPathway(cellchat)
    cellchat <- aggregateNet(cellchat)
    cellchat
}

In [9]:
cellchat_objs <- bplapply(cond, FUN=runCellChat, BPPARAM = MulticoreParam(4))


R_zmq_msg_send errno: 4 strerror: Interrupted system call
[1] "Create a CellChat object from a data matrix"
R_zmq_msg_send errno: 4 strerror: Interrupted system call
Set cell identities for the new CellChat object 
The cell groups used for CellChat analysis are  Epithelial, Fibroblasts, Granulocytes, Macrophages, NK Cells, T Cells 
The number of highly variable ligand-receptor pairs used for signaling inference is 976 
triMean is used for calculating the average gene expression per cell group. 
[1] ">>> Run CellChat on sc/snRNA-seq data <<< [2024-10-28 22:16:20.36779]"
  |======================================================================| 100%
[1] ">>> CellChat inference is done. Parameter values are stored in `object@options$parameter` <<< [2024-10-28 22:20:20.208735]"
The non-filtered cell-cell communication is stored in `object@net$prob.nonFilter` and `object@net$pval.nonFilter`. 


R_zmq_msg_send errno: 4 strerror: Interrupted system call
[1] "Create a CellChat object from a da

In [10]:
saveRDS(file = "individ_cellchat.RData", object = cellchat_objs)

In [11]:
?computeCommunProb

computeCommunProb {CellChat},R Documentation
object,CellChat object
type,"Methods for computing the average gene expression per cell group. By default = ""triMean"", producing fewer but stronger interactions; When setting ‘type = ""truncatedMean""', a value should be assigned to ’trim', producing more interactions."
trim,the fraction (0 to 0.25) of observations to be trimmed from each end of x before the mean is computed
LR.use,A subset of ligand-receptor interactions used in inferring communication network
raw.use,"Whether use the raw data (i.e., 'object@data.signaling') or the smoothed data (i.e., 'object@data.smooth'). Set raw.use = FALSE to use the projected data when analyzing single-cell data with shallow sequencing depth because the projected data could help to reduce the dropout effects of signaling genes, in particular for possible zero expression of subunits of ligands/receptors."
population.size,"Whether consider the proportion of cells in each group across all sequenced cells. Set population.size = FALSE if analyzing sorting-enriched single cells, to remove the potential artifact of population size. Set population.size = TRUE if analyzing unsorted single-cell transcriptomes, with the reason that abundant cell populations tend to send collectively stronger signals than the rare cell populations. Parameters for spatial data analysis:"
distance.use,"Whether to use distance constraints to compute communication probability. Setting 'distance.use = TRUE' indicates that the cell-cell communication probability is inversely proportional to the computed distance. Setting 'distance.use = FALSE' will only filter out interactions between spatially distant regions, but not add distance constraints."
interaction.range,The maximum interaction/diffusion length of ligands (Unit: microns). This hard threshold is used to filter out the connections between spatially distant regions
scale.distance,"A scale or normalization factor for the spatial distances when setting 'distance.use = TRUE'. For example, scale.distance equals 1, 0.1, 0.01, 0.001, 0.11, or 0.011. We choose this values such that the minimum value of the scaled distances is in [1,2]. This value is not necessary when setting 'distance.use = FALSE'. When comparing communication across different CellChat objects, the same scale factor should be used. For a single CellChat analysis, different scale factors will not affect the ranking of the signaling based on their interaction strength."
k.min,The minimum number of interacting cell pairs required for defining spatially proximal cell groups.
